In [ ]:
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
hf_e = h5py.File('../input/ml4sci/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5', 'r')
electron_data_x = hf_e.get('X')
electron_data_y = hf_e.get('y')

In [ ]:
hf_p = h5py.File('../input/ml4sci/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5', 'r')
photon_data_x = hf_p.get('X')
photon_data_y = hf_p.get('y')

In [ ]:
electron_data_x = np.asarray(electron_data_x)
electron_data_y = np.asarray(electron_data_y)

photon_data_x = np.asarray(photon_data_x)
photon_data_y = np.asarray(photon_data_y)

In [ ]:
print(electron_data_x.shape)
print(photon_data_x.shape)

print(electron_data_y.shape)
print(photon_data_y.shape)

In [ ]:
X = np.concatenate((electron_data_x, photon_data_x))
y = np.concatenate((electron_data_y, photon_data_y))

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# X_vis = X[:5]

# fig, axes = plt.subplots(nrows=len(X_vis), ncols=2, figsize=(48, 48))
# for i in range(len(X_vis)):

#     axes[i][0].imshow(X_vis[i,:,:,0])
#     axes[i][1].imshow(X_vis[i,:,:,1])



In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)
X, y = shuffle(X, y, random_state=0) #really random 

In [ ]:
# mean = 0.015
# std = 0.297

In [ ]:
X_train = X[:496900]
X_val = X[496900:497900]
X_test = X[497900:]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
y_train = y[:496900]
y_val = y[496900:497900]
y_test = y[497900:]

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
# model = CreateModel()
# model.to(Config['DEVICE'])

In [ ]:
batch_size = 32
img_size = 32

In [ ]:
# from sklearn.preprocessing import normalize
# normed_matrix = normalize(X, axis=1, norm='l1')

In [ ]:
# X = np.linalg.norm(X, axis = 1, keepdims = True)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator()

# datagen = ImageDataGenerator(
#     featurewise_std_normalization=True,
# )
# datagen.fit(X_train)
# datagen.fit(X_val)
# datagen.fit(X_test)

In [ ]:
# import tensorflow as tf
# model = tf.keras.models.Sequential([

#     tf.keras.layers.Normalization(),
#     tf.keras.layers.Conv2D(32, 2, padding='same', activation='relu', input_shape=(img_size, img_size, 2)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same',),
    

#     tf.keras.layers.Conv2D(64, 2, padding='same',activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
   

#     tf.keras.layers.Conv2D(64, 3,padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  

#     tf.keras.layers.Conv2D(64, 3,padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
#     tf.keras.layers.Conv2D(64, 3,padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(1024, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(128, activation='relu'),
    
    
#     tf.keras.layers.Dense(1, activation='sigmoid')

    
# ])

In [ ]:
# import tensorflow as tf
# model = tf.keras.models.Sequential([

#     tf.keras.layers.Normalization(),
#     tf.keras.layers.Flatten(),  
#     tf.keras.layers.Dense(512, activation='relu',),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(1024, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(128, activation='relu'),
    
    
#     tf.keras.layers.Dense(1, activation='sigmoid')

    
# ])

In [ ]:
image_size = 32
batch_size = 16
n_classes = 1

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 100

patch_size = 7  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [56, 28]  # Size of the dense layers of the final classifier

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = L.Dense(units, activation = tf.nn.gelu)(x)
        x = L.Dropout(dropout_rate)(x)
    return x

In [ ]:
import tensorflow.keras.layers as L


In [ ]:
class Patches(L.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = 'VALID',
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(L.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = L.Dense(units = projection_dim)
        self.position_embedding = L.Embedding(
            input_dim = num_patches, output_dim = projection_dim
        )

    def call(self, patch):
        positions = tf.range(start = 0, limit = self.num_patches, delta = 1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def vision_transformer():
    inputs = L.Input(shape = (image_size, image_size, 2))
    
    # Create patches.
    patches = Patches(patch_size)(inputs)
    
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        
        # Layer normalization 1.
        x1 = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
        
        # Create a multi-head attention layer.
        attention_output = L.MultiHeadAttention(
            num_heads = num_heads, key_dim = projection_dim, dropout = 0.1
        )(x1, x1)
        
        # Skip connection 1.
        x2 = L.Add()([attention_output, encoded_patches])
        
        # Layer normalization 2.
        x3 = L.LayerNormalization(epsilon = 1e-6)(x2)
        
        # MLP.
        x3 = mlp(x3, hidden_units = transformer_units, dropout_rate = 0.1)
        
        # Skip connection 2.
        encoded_patches = L.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
    representation = L.Flatten()(representation)
    representation = L.Dropout(0.5)(representation)
    
    # Add MLP.
    features = mlp(representation, hidden_units = mlp_head_units, dropout_rate = 0.5)
    
    # Classify outputs.
    logits = L.Dense(n_classes)(features)
    logits_ = tf.keras.activations.sigmoid(logits)
    
    # Create the model.
    model = tf.keras.Model(inputs = inputs, outputs = logits_)
    
    return model

In [ ]:
from tensorflow.keras.optimizers import Adam
model = vision_transformer()
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['AUC']
)

In [ ]:
call_back = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

In [ ]:
labels = [0,1]

In [ ]:

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=datagen.flow(X_val, y_val,
    batch_size=batch_size),
    epochs = 100,
    callbacks = [call_back]
)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

In [ ]:
acc = history.history['auc']
val_acc = history.history['val_auc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training & validation accuracy')
plt.figure()

plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title('Training & validation loss')